### **SCD TYPE - 1**

In [0]:
%sql
CREATE TABLE datamodeling.default.scdtyp1_source
(
  prod_id INT,
  prod_name STRING,
  prod_cat STRING,
  processDate DATE
)

In [0]:
%sql
INSERT INTO datamodeling.default.scdtyp1_source
VALUES
(1,'prod1','cat1',CURRENT_DATE()),
(2,'prod2','cat2',CURRENT_DATE()),
(3,'prod3','cat3',CURRENT_DATE())

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
CREATE TABLE datamodeling.gold.scdtyp1_table
(
  prod_id INT,
  prod_name STRING,
  prod_cat STRING,
  processDate DATE
)

In [0]:
spark.sql("select * from datamodeling.default.scdtyp1_source").createOrReplaceTempView("src")

In [0]:
%sql
MERGE INTO datamodeling.gold.scdtyp1_table AS trg
USING src
ON src.prod_id = trg.prod_id
WHEN MATCHED AND src.processDate >= trg.processDate THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,3,0,0


In [0]:
%sql
SELECT * FROM datamodeling.gold.scdtyp1_table

prod_id,prod_name,prod_cat,processDate
1,prod1,cat1,2025-07-16
3,prod3,cat3,2025-07-16
2,prod2,cat2,2025-07-16


In [0]:
%sql
UPDATE datamodeling.default.scdtyp1_source SET prod_cat = 'newcategory'
WHERE prod_id = 3

num_affected_rows
1


# **SCD TYPE 2**

### **Source Creation**

In [0]:
%sql
CREATE TABLE datamodeling.default.scdtyp2_source
(
  prod_id INT,
  prod_name STRING,
  prod_cat STRING,
  processDate DATE
)

In [0]:
%sql
INSERT INTO datamodeling.default.scdtyp2_source
VALUES
(1,'prod1','cat1',CURRENT_DATE()),
(2,'prod2','cat2',CURRENT_DATE()),
(3,'prod3','cat3',CURRENT_DATE())

num_affected_rows,num_inserted_rows
3,3


### **Target Table**

In [0]:
%sql
CREATE TABLE datamodeling.gold.scdtype2_table
(
  prod_id INT,
  prod_name STRING,
  prod_cat STRING,
  processDate DATE,
  start_date DATE,
  end_date DATE,
  is_current STRING
)

**Adding additional 3 columns for SCD Type - 2**

In [0]:
spark.sql("""SELECT *,
        current_timestamp as start_date,
        CAST('3000-01-01' AS TIMESTAMP) as end_date,
        'Y' as is_current
FROM datamodeling.default.scdtyp2_source""").createOrReplaceTempView("src")

### **SCD TYPE 2 MEGE CONDISTIONS**

**MERGE-1** : This command will check if we have any data in the target table that is updated in the source, and will mark it as expired. 

In [0]:
%sql
MERGE INTO datamodeling.gold.scdtype2_table AS trg
USING src 
ON src.prod_id = trg.prod_id
AND trg.is_current = 'Y'

-- When We have New Data With Updates
WHEN MATCHED AND (
  src.prod_cat <> trg.prod_cat OR
  src.processDate <> trg.processDate OR
  src.prod_name <> trg.prod_name
) THEN 
  UPDATE SET 
    trg.end_date = current_timestamp(),
    trg.is_current = 'N'

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0


**MERGE-2** : This command will bring all the non-expired commands bcz we have filter of "is_current = 'Y'". So, this will not bring the updated records as well bcz previous MERGE command marked it as expired. So all the new records [including updated] will be inserted in this MERGE.

In [0]:
%sql
MERGE INTO datamodeling.gold.scdtype2_table AS trg
USING src 
ON trg.prod_id = src.prod_id 
AND trg.is_current = 'Y' 

WHEN NOT MATCHED THEN INSERT 
(
  prod_id,
  prod_name,
  prod_cat,
  processDate,
  start_date,
  end_date,
  is_current
) VALUES (
  src.prod_id,
  src.prod_name,
  src.prod_cat,
  src.processDate,
  src.start_date,
  src.end_date,
  src.is_current
)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,0,0,3


In [0]:
%sql
SELECT * FROM datamodeling.gold.scdtype2_table

prod_id,prod_name,prod_cat,processDate,start_date,end_date,is_current
1,prod1,cat1,2025-07-16,2025-07-16,3000-01-01,Y
2,prod2,cat2,2025-07-16,2025-07-16,3000-01-01,Y
3,prod3,cat3,2025-07-16,2025-07-16,3000-01-01,Y
